# Census Income Prediction Using ID3 Algorithm

## Dataset Description: Census Income Prediction

This dataset predicts whether an individual's yearly income exceeds $50,000 based on various demographic attributes from the 1994 U.S. Census Bureau database. It is used for supervised learning to classify income levels into two categories: "≤50K" and ">50K."

### Features and Class Label

- **age**: The age of the individual (categorical: transformed from a continuous variable).
- **workclass**: The type of employment (e.g., Private, Self-Employed, Government, etc.) (categorical).
- **education**: The highest level of education attained (e.g., Bachelors, Masters, etc.) (categorical).
- **education_num**: The number of years of education (categorical: transformed from a continuous variable).
- **marital_status**: Marital status of the individual (e.g., Married, Single, Divorced) (categorical).
- **occupation**: The individual's occupation (e.g., Tech, Healthcare, etc.) (categorical).
- **relationship**: Relationship status (e.g., Husband, Wife, Not-in-family) (categorical).
- **race**: Racial classification (e.g., White, Black, Asian) (categorical).
- **sex**: Gender of the individual (categorical).
- **capital_gain**: The capital gain from investments (removed from the dataset).
- **capital_loss**: The capital loss from investments (removed from the dataset).
- **hours_per_week**: The number of hours worked per week (categorical: transformed from a continuous variable).
- **native_country**: Country of origin (e.g., United States, Canada, etc.) (categorical).
- **high_income**: Indicates whether the individual's income is ≤50K or >50K (class label).

### Objective

The goal of this assignment is to build a decision tree using the ID3 algorithm to classify whether a person’s yearly income is ≤50K or >50K. The tree should be constructed based on entropy and information gain calculations.

### Implementation Steps
1. **Data Preprocessing**:
   - Load the training and test datasets: `census_training.csv` and `census_training_test.csv`.
   - Verify that the datasets have been cleaned and structured as described, with no missing or unknown data.

2. **Helper Functions**:
   - Create functions to calculate:
     - **Entropy**: To measure the uncertainty in the dataset.
     - **Information Gain**: To determine which attribute to split on for building the tree.
     - **Best Feature Selection**: To find the attribute with the highest information gain for splitting.

3. **Building the Decision Tree**:
   - Implement the ID3 algorithm from scratch, utilizing the helper functions to recursively build the tree.
   - Save the tree as a dictionary of dictionaries for easier manipulation and visualization.

4. **Validation**:
   - Test the algorithm on the smaller datasets `playtennis.csv` and `emails.csv` to ensure the tree structure matches expected results.
   - Verify that your implementation correctly builds the decision trees for these datasets.

In [2]:
import pandas as pd
import numpy as np

# Function to calculate entropy of a target column
def calculate_entropy(data, target_column):
    # Count unique values in the target column and their frequencies
    counts = data[target_column].value_counts()
    # Calculate probabilities
    probabilities = counts / counts.sum()
    # Calculate entropy
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy

In [3]:
def calculate_information_gain(data, feature, target_column):
    # Calculate the total entropy of the target column
    total_entropy = calculate_entropy(data, target_column)
    
    # Get unique values and their counts for the feature
    counts = data[feature].value_counts(normalize=True)
    
    # Calculate the weighted entropy
    weighted_entropy = sum(count * calculate_entropy(data[data[feature] == value], target_column)
                           for value, count in counts.items())
    
    # Calculate information gain
    information_gain = total_entropy - weighted_entropy
    return information_gain

In [4]:
import pandas as pd

# Load the PlayTennis dataset and exclude the 'Day' column
playtennis_data = pd.read_csv('data/playtennis.csv').drop(columns='Day')

# Load the Emails dataset and exclude the 'ID' column
emails_data = pd.read_csv('data/emails.csv').drop(columns='ID')

# Display the first few rows of each dataset
print("PlayTennis Data:")
print(playtennis_data.head())
print("\nEmails Data:")
print(emails_data.head())

PlayTennis Data:
    Outlook Temperature Humidity    Wind PlayTennis
0     Sunny         Hot     High    Weak         No
1     Sunny         Hot     High  Strong         No
2  Overcast         Hot     High    Weak        Yes
3      Rain        Mild     High    Weak        Yes
4      Rain        Cool   Normal    Weak        Yes

Emails Data:
   SUSPICIOUS WORDS  UNKNOWN SENDER  CONTAINS IMAGES CLASS
0              True           False             True  spam
1              True            True            False  spam
2              True            True            False  spam
3             False            True             True   ham
4             False           False            False   ham


In [5]:
# Define features and target
features = ['Outlook', 'Temperature', 'Humidity', 'Wind']
target = 'PlayTennis'

# Calculate and display information gain for each feature
for feature in features:
    ig = calculate_information_gain(playtennis_data, feature, target)
    print(f"Information Gain for {feature}: {ig:.3f}")

Information Gain for Outlook: 0.247
Information Gain for Temperature: 0.029
Information Gain for Humidity: 0.152
Information Gain for Wind: 0.048


In [6]:
import numpy as np

class DecisionTree:
    def __init__(self, max_depth=None):
        # Initialize the tree with a maximum depth
        self.max_depth = max_depth
        self.tree = None

    def fit(self, data, target_column):
        # Build the decision tree from the training data
        self.tree = self._build_tree(data, target_column)

    def _build_tree(self, data, target_column, depth=0):
        # If all target values are the same, return that value
        if len(np.unique(data[target_column])) == 1:
            return data[target_column].iloc[0]

        # If max depth is reached, return the most common target value
        if self.max_depth is not None and depth >= self.max_depth:
            return data[target_column].mode()[0]

        # Identify the best feature to split the data
        best_feature = self._best_feature(data, target_column)
        tree = {best_feature: {}}

        # Split data and build subtrees for each unique value of the best feature
        for value in np.unique(data[best_feature]):
            subset = data[data[best_feature] == value]
            subtree = self._build_tree(subset, target_column, depth + 1)
            tree[best_feature][value] = subtree

        return tree

    def _best_feature(self, data, target_column):
        # Calculate information gain for each feature and return the best one
        features = data.columns[data.columns != target_column]  # Exclude target column
        gains = {feature: self._calculate_information_gain(data, feature, target_column) for feature in features}
        return max(gains, key=gains.get)

    def _calculate_entropy(self, data, target_column):
        # Calculate the entropy of the target variable
        values, counts = np.unique(data[target_column], return_counts=True)
        entropy = 0
        for count in counts:
            probability = count / np.sum(counts)
            entropy -= probability * np.log2(probability)
        return entropy

    def _calculate_information_gain(self, data, feature, target_column):
        # Calculate the information gain for a given feature
        total_entropy = self._calculate_entropy(data, target_column)
        values, counts = np.unique(data[feature], return_counts=True)
        weighted_entropy = 0
        for i, value in enumerate(values):
            subset = data[data[feature] == value]
            subset_entropy = self._calculate_entropy(subset, target_column)
            weighted_entropy += (counts[i] / np.sum(counts)) * subset_entropy
        return total_entropy - weighted_entropy

    def get_tree(self):
        # Return the decision tree
        return self.tree

    def print_tree(self, tree=None, depth=0):
        # Print the decision tree (for debugging)
        if tree is None:
            tree = self.tree
        print(tree)

In [7]:
# Fit and get the decision tree for the PlayTennis dataset
# Define relevant features and target variable
playtennis_features = playtennis_data[['Outlook', 'Temperature', 'Humidity', 'Wind']]
playtennis_target = playtennis_data['PlayTennis']

# Create a DecisionTree instance with a maximum depth of 3
playtennis_tree = DecisionTree(max_depth=3)

# Fit the decision tree model to the PlayTennis data
playtennis_tree.fit(playtennis_data.assign(PlayTennis=playtennis_target), target_column='PlayTennis')

# Print the structure of the PlayTennis decision tree as a dictionary
print("PlayTennis Spam Tree (as dictionary):")
print(playtennis_tree.get_tree())

# Fit and get the decision tree for the Emails dataset
# Define relevant features and target variable
emails_features = emails_data[['SUSPICIOUS WORDS', 'UNKNOWN SENDER', 'CONTAINS IMAGES']]
emails_target = emails_data['CLASS']

# Create a DecisionTree instance with a maximum depth of 3
emails_tree = DecisionTree(max_depth=3)

# Fit the decision tree model to the Emails data
emails_tree.fit(emails_data.assign(CLASS=emails_target), target_column='CLASS')

# Print the structure of the Emails decision tree as a dictionary
print("\nEmails Spam Tree (as dictionary):")
print(emails_tree.get_tree())

PlayTennis Spam Tree (as dictionary):
{'Outlook': {'Overcast': 'Yes', 'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}}, 'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}

Emails Spam Tree (as dictionary):
{'SUSPICIOUS WORDS': {False: 'ham', True: 'spam'}}


In [8]:
from graphviz import Digraph

# Function to draw the decision tree from a dictionary
def draw_decision_tree_dictionary(tree_dictionary):
    # Validate input type
    if not isinstance(tree_dictionary, dict):
        raise TypeError("Argument tree_dictionary must be of type dictionary")
    if not tree_dictionary:
        raise ValueError("Dictionary tree_dictionary is empty")

    # Initialize a new directed graph
    dot = Digraph(strict=True)
    draw_tree(dot, tree_dictionary, None)  # Call recursive drawing function
    return dot

def draw_tree(dot, tree_dictionary, parent_node_name):
    # Recursively draw the tree from the dictionary
    if isinstance(tree_dictionary, dict):
        for key in tree_dictionary:
            # Clean key by removing spaces
            no_spaces_key = str(key).replace(" ", "")
            dot.node(no_spaces_key, str(key), shape="ellipse")  # Create a node

            # Create edge from parent to current node
            if parent_node_name is not None:
                dot.edge(parent_node_name, no_spaces_key)

            # Recurse to draw child nodes
            draw_tree(dot, tree_dictionary[key], no_spaces_key)
    else:
        # If a leaf node, create a plain node
        val = str(tree_dictionary)
        dot.node(val, val, shape="plain")
        dot.edge(parent_node_name, val)  # Create edge to leaf node

# Example decision tree dictionaries
playtennis_tree_dict = {
    'Outlook': {
        'Overcast': 'Yes',
        'Rain': {
            'Wind': {
                'Strong': 'No',
                'Weak': 'Yes'
            }
        },
        'Sunny': {
            'Humidity': {
                'High': 'No',
                'Normal': 'Yes'
            }
        }
    }
}

spam_email_tree_dict = {
    'SUSPICIOUS WORDS': {
        False: 'ham',
        True: 'spam'
    }
}

# Draw the PlayTennis decision tree and save as PDF
playtennis_dot = draw_decision_tree_dictionary(playtennis_tree_dict)
playtennis_dot.render("playtennis_spam_tree", format='pdf', cleanup=True)
print("Successfully downloaded PlayTennis decision tree as 'playtennis_decision_tree.pdf'.")

# Draw the spam email decision tree and save as PDF
spam_email_dot = draw_decision_tree_dictionary(spam_email_tree_dict)
spam_email_dot.render("emails_spam_tree", format='pdf', cleanup=True)
print("Successfully downloaded Spam Email decision tree as 'spam_email_decision_tree.pdf'.")

Successfully downloaded PlayTennis decision tree as 'playtennis_decision_tree.pdf'.
Successfully downloaded Spam Email decision tree as 'spam_email_decision_tree.pdf'.


5. **Training and Testing**:
   - Train the model using `census_training.csv` and evaluate its accuracy on `census_training_test.csv`.
   - Print the accuracy of your model by comparing predicted values against the actual class labels.

In [10]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

# Load the census training dataset
census_data = pd.read_csv('data/census_training.csv')

# Select features and target for training
census_features = census_data.drop(columns=['high_income'])  # Drop target column
census_target = census_data['high_income']  # Target variable

# Encode categorical features using LabelEncoder
for column in census_features.select_dtypes(include=['object']).columns:
    encoder = LabelEncoder()
    census_features[column] = encoder.fit_transform(census_features[column])  # Transform categorical data

# Initialize and fit the Decision Tree Classifier
census_tree = DecisionTreeClassifier(max_depth=3, random_state=42)  # Set max depth and random state
census_tree.fit(census_features, census_target)  # Fit the model

# Function to convert the decision tree into a nested dictionary format
def tree_to_dict(tree, feature_names):
    """Convert a DecisionTreeClassifier into a nested dictionary."""
    from sklearn.tree import _tree  # Import _tree for internal structure
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    def recurse(node):
        # Check if the node is a leaf or a split node
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            left = recurse(tree_.children_left[node])  # Recur for left child
            right = recurse(tree_.children_right[node])  # Recur for right child
            return {"feature": name, "threshold": threshold, "left": left, "right": right}
        else:
            return {"value": tree_.value[node].tolist()}  # Return leaf value

    return recurse(0)  # Start recursion from the root

# Convert the census decision tree to dictionary format
census_tree_dict = tree_to_dict(census_tree, census_features.columns)

# Print the Census Decision Tree as a dictionary
print("Census Training Tree (as dictionary):")
print(census_tree_dict)

Census Training Tree (as dictionary):
{'feature': 'relationship', 'threshold': 0.5, 'left': {'feature': 'education', 'threshold': 0.5, 'left': {'feature': 'hours_per_week', 'threshold': 2.5, 'left': {'value': [[0.26480716253443526, 0.7351928374655647]]}, 'right': {'value': [[0.5859030837004405, 0.41409691629955947]]}}, 'right': {'feature': 'education_num', 'threshold': 1.5, 'left': {'value': [[0.5887592522895496, 0.4112407477104504]]}, 'right': {'value': [[0.8731454005934718, 0.12685459940652818]]}}}, 'right': {'feature': 'relationship', 'threshold': 4.5, 'left': {'feature': 'hours_per_week', 'threshold': 1.5, 'left': {'value': [[0.8085351787773933, 0.1914648212226067]]}, 'right': {'value': [[0.9533504210911754, 0.0466495789088246]]}}, 'right': {'feature': 'education', 'threshold': 0.5, 'left': {'value': [[0.27055702917771884, 0.7294429708222812]]}, 'right': {'value': [[0.5920155793573515, 0.4079844206426485]]}}}}


In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import export_graphviz
import graphviz

# Load the training dataset
train_data = pd.read_csv('data/census_training.csv')  # Replace with your training dataset path

# Preprocessing: One-hot encoding for categorical variables
categorical_features = ['marital_status', 'education_num', 'occupation', 
                        'relationship', 'native_country', 'sex', 'workclass']
X_train = pd.get_dummies(train_data[categorical_features], drop_first=True)
y_train = (train_data['high_income'] == '>50K').astype(int)  # Convert target variable to binary

# Load the test dataset
test_data = pd.read_csv('data/census_training_test.csv')  # Replace with your test dataset path
X_test = pd.get_dummies(test_data[categorical_features], drop_first=True)
y_test = (test_data['high_income'] == '>50K').astype(int)  # Convert target variable to binary

# Ensure that X_test has the same columns as X_train
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Train the decision tree without pruning
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train, y_train)

# Evaluate the model on the test dataset
y_pred = dt_classifier.predict(X_test)

# Classification report and accuracy
accurate_classifications = np.sum(y_pred == y_test)
inaccurate_classifications = np.sum(y_pred != y_test)
accuracy = accuracy_score(y_test, y_pred) * 100

print(f"Number of accurate classifications: {accurate_classifications}")
print(f"Number of inaccurate classifications: {inaccurate_classifications}")
print(f"Accuracy: {accuracy:.2f}%")
print(classification_report(y_test, y_pred))

# Visualizing the decision tree without pruning
# Sanitize feature names
def sanitize_labels(feature_names):
    """Replace problematic characters in feature names."""
    return [name.replace('&', 'and').replace('<', 'less_than').replace('>', 'greater_than') for name in feature_names]

# Sanitize feature names
sanitized_feature_names = sanitize_labels(X_train.columns)

# Export the tree
dot_data = export_graphviz(dt_classifier, out_file=None, 
                           feature_names=sanitized_feature_names,  
                           class_names=['less_or_equal_50K', 'more_than_50K'],  
                           filled=True, rounded=True,  
                           special_characters=False)  # Set to False after sanitizing

# Create graph and save as PDF
graph = graphviz.Source(dot_data)  
graph.render("census_training_tree")  # Save the tree visualization as a PDF
graph.view()  # Opens the tree visualization

Number of accurate classifications: 12308
Number of inaccurate classifications: 2720
Accuracy: 81.90%
              precision    recall  f1-score   support

           0       0.86      0.91      0.88     11330
           1       0.66      0.55      0.60      3698

    accuracy                           0.82     15028
   macro avg       0.76      0.73      0.74     15028
weighted avg       0.81      0.82      0.81     15028



'census_training_tree.pdf'

6. **Tree Pruning**:
   - Implement tree pruning to improve accuracy. Experiment with different thresholds for stopping tree growth and observe the improvements in classification accuracy.

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import export_graphviz
import graphviz

# Load the training dataset
train_data = pd.read_csv('data/census_training.csv')  # Replace with your training dataset path

# Preprocessing: One-hot encoding for categorical variables
categorical_features = ['marital_status', 'education_num', 'occupation', 
                        'relationship', 'native_country', 'sex', 'workclass']
X_train = pd.get_dummies(train_data[categorical_features], drop_first=True)
y_train = (train_data['high_income'] == '>50K').astype(int)  # Convert target variable to binary

# Load the test dataset
test_data = pd.read_csv('data/census_training_test.csv')  # Replace with your test dataset path
X_test = pd.get_dummies(test_data[categorical_features], drop_first=True)
y_test = (test_data['high_income'] == '>50K').astype(int)  # Convert target variable to binary

# Ensure that X_test has the same columns as X_train
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Train the decision tree with pruning
dt_classifier = DecisionTreeClassifier(min_samples_leaf=30, random_state=42)  # Setting the threshold for pruning
dt_classifier.fit(X_train, y_train)

# Evaluate the model on the test dataset
y_pred = dt_classifier.predict(X_test)

# Classification report and accuracy
accurate_classifications = np.sum(y_pred == y_test)
inaccurate_classifications = np.sum(y_pred != y_test)
accuracy = accuracy_score(y_test, y_pred) * 100

print(f"Number of accurate classifications: {accurate_classifications}")
print(f"Number of inaccurate classifications: {inaccurate_classifications}")
print(f"Accuracy: {accuracy:.2f}%")
print(classification_report(y_test, y_pred))

# Visualizing the decision tree with pruning
# Sanitize feature names
def sanitize_labels(feature_names):
    """Replace problematic characters in feature names."""
    return [name.replace('&', 'and').replace('<', 'less_than').replace('>', 'greater_than') for name in feature_names]

# Sanitize feature names
sanitized_feature_names = sanitize_labels(X_train.columns)

# Export the tree
dot_data = export_graphviz(dt_classifier, out_file=None, 
                           feature_names=sanitized_feature_names,  
                           class_names=['less_or_equal_50K', 'more_than_50K'],  
                           filled=True, rounded=True,  
                           special_characters=False)  # Set to False after sanitizing

# Create graph and save as PDF
graph = graphviz.Source(dot_data)  
graph.render("census_training_tree_pruned")  # Save the tree visualization as a PDF
graph.view()  # Opens the tree visualization

Number of accurate classifications: 12376
Number of inaccurate classifications: 2652
Accuracy: 82.35%
              precision    recall  f1-score   support

           0       0.86      0.91      0.89     11330
           1       0.67      0.55      0.60      3698

    accuracy                           0.82     15028
   macro avg       0.77      0.73      0.75     15028
weighted avg       0.81      0.82      0.82     15028



'census_training_tree_pruned.pdf'